In [43]:
from PIL import Image
import os
import numpy as np
import pandas as pd
import csv

In [44]:
alex_images_path = "Alex_Kelly_Pics/Alex"
kelly_images_path = "Alex_Kelly_Pics/Kelly"
test_images_path = "Alex_Kelly_Pics/TestSet"

In [45]:
def sorted_image_names(path):
    return sorted(os.listdir(path), key=lambda x: int(''.join(filter(str.isdigit, x))))

In [46]:
alex_image_names = sorted_image_names(alex_images_path)
kelly_image_names = sorted_image_names(kelly_images_path)
test_image_names = sorted_image_names(test_images_path)

image_names = alex_image_names + kelly_image_names

In [47]:
def load_images_from_folder(folder, resize_shape=(64, 64)):
    images = []
    image_names = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename)).convert('RGB')
        img = img.resize(resize_shape)
        if img is not None:
            img_array = np.array(img)
            images.append(img_array.flatten())
    return images

In [ ]:
alex_images = load_images_from_folder(alex_images_path)
kelly_images = load_images_from_folder(kelly_images_path)
test_images = load_images_from_folder(test_images_path)


alex_images_df = pd.DataFrame(alex_images, index=alex_image_names)
kelly_images_df = pd.DataFrame(kelly_images, index=kelly_image_names)
test_images_df = pd.DataFrame(test_images, index=test_image_names)

In [ ]:
alex_kelly_images_df = pd.concat([alex_images_df, kelly_images_df])
alex_kelly_images_df.head(5)